In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Please support by upvoting if you like the content

In [ ]:
!pip install verstack

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
#from verstack.stratified_continuous_split import scsplit
from sklearn.multioutput import MultiOutputRegressor

import warnings
warnings.filterwarnings("ignore")

In [ ]:
trainset = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
testset = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

# print(trainset.info())
# print(trainset.describe())

# print(testset.info())
# print(testset.describe())

features = testset.columns
targets = ["target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"]

# Preprocess - Categorize Date and time for train and test set

In [ ]:
#trainset[['date','time']] = trainset.date_time.str.split(expand=True)
def categorize_time_of_day(dataframe):
    dataframe['date_and_time'] = pd.to_datetime(dataframe.date_time)
    dataframe['month'] = dataframe["date_and_time"].dt.month
    dataframe['week'] = dataframe["date_and_time"].dt.day // 7 + 1
    dataframe['sme'] = dataframe.relative_humidity / dataframe.absolute_humidity
    
    dataframe['day'] = dataframe.date_time.apply(lambda x: x.split(" ")[-1].split(":")[0]).astype(int)
    dataframe.loc[(dataframe['day'] > 4) & (dataframe['day'] < 6), 'time_of_day'] = "dawn"
    dataframe.loc[(dataframe['day'] >= 6) & (dataframe['day'] <= 10), 'time_of_day'] = "morning"
    dataframe.loc[(dataframe['day'] > 10) & (dataframe['day'] <= 13), 'time_of_day'] = "noon"
    dataframe.loc[(dataframe['day'] > 13) & (dataframe['day'] <= 15), 'time_of_day'] = "afternoon"
    dataframe.loc[(dataframe['day'] > 15) & (dataframe['day'] <= 18), 'time_of_day'] = "evening"
    dataframe.loc[(dataframe['day'] > 18) & (dataframe['day'] < 23), 'time_of_day'] = "night"
    dataframe.loc[(dataframe['day'] > 22) | (dataframe['day'] <= 4), 'time_of_day'] = "midnight"

#     dataframe.loc[(dataframe['day'] >= 6) & (dataframe['day'] <= 11), 'time_of_day'] = "morning"
#     dataframe.loc[(dataframe['day'] > 11) & (dataframe['day'] <= 17), 'time_of_day'] = "afternoon"
#     dataframe.loc[(dataframe['day'] > 17) & (dataframe['day'] <= 22), 'time_of_day'] = "evening"
#     dataframe.loc[(dataframe['day'] > 22) | (dataframe['day'] <= 5), 'time_of_day'] = "night"
    
    dataframe.loc[dataframe['deg_C'] <= 10, 'temp_cat'] = 10
    dataframe.loc[(dataframe['deg_C'] > 10) & (dataframe['deg_C'] <= 20), 'temp_cat'] = 20
    dataframe.loc[(dataframe['deg_C'] > 20) & (dataframe['deg_C'] <= 30), 'temp_cat'] = 30
    dataframe.loc[(dataframe['deg_C'] > 30) | (dataframe['deg_C'] <= 40), 'temp_cat'] = 40
    dataframe.loc[dataframe['deg_C'] > 40, 'temp_cat'] = 50
    dataframe.temp_cat = dataframe.temp_cat.astype(int)

    dataframe = dataframe.drop(['day', 'date_time'], axis=1)
    dataframe.loc[dataframe.date_and_time.dt.dayofweek < 5, 'weekday'] = 1
    dataframe.loc[dataframe.date_and_time.dt.dayofweek == 5, 'weekday'] = 2
    dataframe.loc[dataframe.date_and_time.dt.dayofweek > 5, 'weekday'] = 3
    dataframe.weekday = dataframe.weekday.astype(int)
    
    return dataframe

trainset1 = categorize_time_of_day(trainset)
testset1 = categorize_time_of_day(testset)

# Data distribution - comparison of data between train and test set


In [ ]:
sns.set()
features = features[1:]
#print(features)

fig, ax = plt.subplots(figsize=(30, 60))
for count, feature in enumerate(features):
    plt.subplot(len(features), 2, count+1)
    sns.distplot(trainset[feature], color="blue", kde=True)
    sns.distplot(testset[feature], color="red", kde=True)
plt.show()

# Pollution level during a part of day - visualisation

The below graphs gives following insights:
* Carbon monoxide pollutant is clustered more between temperatures 8 to 30 degree celcius
* Benzene pollutant is clustered more between temperatures 8 to 30 degree celcius
* Nitrogen oxides pollutant is clustered more between temperatures 8 to 35 degree celcius
* Nitrogen oxides data has more noise comparing to other two

In [ ]:
for target in targets:
    fig, ax = plt.subplots(figsize=(20, 10))
    sns.scatterplot(data=trainset, x="deg_C", y=target, hue="time_of_day")
    plt.show()

# Pollution leve during weekdays and weekends - visualisation

**The key takeaway from below graphs is the pollution level is high during weekdays - mon to fri compared to weekends - sat and sun**

In [ ]:
for target in targets:
    fig, ax = plt.subplots(figsize=(17, 8))
    sns.scatterplot(data=trainset1, x="deg_C", y=target, hue="weekday")
    plt.show()

# Temperature impact on pollution level - visualisation

**The plot shows, raise in temperature has minimal to no effect on pollutant level.
Pollutant level is very dense in 8 to 35 degree celcius**

In [ ]:
fig, ax = plt.subplots(3, figsize=(20, 20))
sns.scatterplot(data=trainset, x="deg_C", y=targets[0], ax=ax[0], color="blue")
sns.scatterplot(data=trainset, x="deg_C", y=targets[1],ax=ax[1], color="red")
sns.scatterplot(data=trainset, x="deg_C", y=targets[2],ax=ax[2], color="green")
plt.show()

# Level of pollutants over a time period - visualisation

**The graph shows that the pollution level dropped during the month of August and incresed on upcoming months. The first half of the year has less pollutant level compared to the second half**

In [ ]:
fig, ax = plt.subplots(3, figsize=(20, 20))
sns.scatterplot(data=trainset1, x="date_and_time", y=targets[0], ax=ax[0], color="blue")
sns.scatterplot(data=trainset1, x="date_and_time", y=targets[1], ax=ax[1], color="red")
sns.scatterplot(data=trainset1, x="date_and_time", y=targets[2], ax=ax[2], color="green")
plt.show()

# Correlation matrix - visualisation

**Syntheic columns weekday/weekend and time_of_day has decent effect on the pollutant level, which would be useful while developing a ML model**

In [ ]:
def convert_to_numeric(dataframe):
    dataframe['time_of_day'] = dataframe['time_of_day'].map({'morning': 1, 'noon': 2, 'afternoon': 3, 'evening': 4, 'night': 5, 'midnight': 6, 'dawn': 0}).astype(int)
    #dataframe['time_of_day'] = dataframe['time_of_day'].map({'morning': 0, 'afternoon': 1, 'evening': 2, 'night': 3}).astype(int)
    return dataframe

In [ ]:
trainset1.week.unique()

In [ ]:
trainset1 = convert_to_numeric(trainset1)
corr_matrix = trainset1.corr()
corr_matrix

In [ ]:
# reused from notebooke: https://www.kaggle.com/maximkazantsev/tps-07-21-eda-catboost
df = trainset1.copy()
df = pd.concat([df[targets], df.drop(targets, axis=1)], axis=1).corr().round(2)

# Mask to hide upper-right part of plot as it is a duplicate
mask = np.zeros_like(df)
mask[np.triu_indices_from(mask)] = True

# Making a plot
plt.figure(figsize=(12,12))
ax = sns.heatmap(df, annot=True, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})
ax.set_title("Original dataset correlation heatmap", fontsize=17)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor", weight="bold")
plt.setp(ax.get_yticklabels(), weight="bold")
plt.show();

# Data processing

In [ ]:
testset1 = convert_to_numeric(testset1)

In [ ]:
input_features = [column for column in trainset1.columns if column not in targets]
print(input_features)

In [ ]:
trainset1.info()

In [ ]:
fit_params = {'cat_features':['time_of_day', 'month', 'weekday', 'week', 'temp_cat'], 'verbose':0}

In [ ]:
cb_params = [] 
for itr, target in enumerate(targets):
    model = CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE')
    param_dist = {
                    "learning_rate": np.linspace(0.001,0.1,3), 
                    "bagging_temperature": np.linspace(4, 12), 
                    "leaf_estimation_method": ("Gradient", "Newton"),
                    "depth": np.linspace(5, 12),
                    "num_trees": randint(100, 1000) 
                 }
    rscv = RandomizedSearchCV(model , param_dist, n_jobs=-1, cv=8)
    rscv.fit(trainset1.loc[:, input_features], trainset1.loc[:, target], **fit_params)
    print(rscv.best_params_)
    print(rscv.best_score_)
    
    cb_params.append(rscv.best_params_)

In [ ]:
# cb_params = [{'bagging_temperature': 6.612244897959183, 'depth': 10.0, 'leaf_estimation_method': 'Gradient', 'learning_rate': 0.1, 'num_trees': 923},
#              {'bagging_temperature': 9.061224489795919, 'depth': 6.0, 'leaf_estimation_method': 'Gradient', 'learning_rate': 0.0505, 'num_trees': 275},
#              {'bagging_temperature': 8.73469387755102, 'depth': 11.0, 'leaf_estimation_method': 'Gradient', 'learning_rate': 0.1, 'num_trees': 129}]

In [ ]:
feature_importance = pd.DataFrame()
for i in range(len(targets)):
    model = CatBoostRegressor(grow_policy='Lossguide', loss_function='RMSE', eval_metric='RMSE', **cb_params[i])
    model.fit(trainset1.loc[:, input_features], trainset1.loc[:, targets[i]], cat_features=fit_params['cat_features'], verbose=0)
    pred = model.predict(testset1.loc[:, input_features])
    submission[targets[i]] = pred
    
    tmp = pd.DataFrame()
    tmp['features'] = input_features
    tmp['value'] = model.feature_importances_
    tmp['target'] = targets[i]
    
    feature_importance = pd.concat([feature_importance, tmp])

In [ ]:
plt.figure(figsize=(22, 12))
sns.barplot(data=feature_importance, x='features', y='value', hue='target')

# Testset pollutant prediction - visualisation

In [ ]:
columns = ['date_time', 'target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
trainset2 = trainset[columns]
trainset2['data'] = 'Training set'
testset2 = submission[columns]
testset2['data'] = 'Test set'
combined = pd.concat([trainset2, testset2])
combined['date_time'] = pd.to_datetime(combined.date_time)

In [ ]:
for target in targets:
    fig, ax = plt.subplots(figsize=(17, 8))
    sns.lineplot(data=combined, x="date_time", y=target, hue="data", style="data")
    plt.show()

In [ ]:
submission.to_csv("submission.csv", index=False)